# Recherche de production solaire

In [ ]:
import pandas as pd
import requests
import numpy as np
import datetime as dt

df = pd.read_csv("data/prod_solaire.csv")

df

In [ ]:
df.loc[(df.iloc[:, 1] < 100)  & (df.iloc[:, 1] > 0)]
types = ["hydro", "eolienne", "solar"]
print(f"Please enter any of types in this list : {types} ")

In [ ]:
df.describe()

In [ ]:
df_missing = df.isnull().sum()

display(df_missing)

df_ext = df.sort_values(by=["prod_solaire"], ascending=True)


In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.loc[df["prod_solaire"] > 0].sort_values(by="prod_solaire")

df["date"].dt.dayofweek.value_counts()



In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 43.6109,
	"longitude": 3.8763,
	"start_date": "2016-09-01",
	"end_date": "2025-09-25",
	"hourly": ["global_tilted_irradiance", "temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_50m", "wind_direction_10m"],
	"tilt": 35,
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_global_tilted_irradiance = hourly.Variables(0).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_50m = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["wind_speed_50m"] = hourly_wind_speed_50m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_global_tilted_irradiance)
gti_df = hourly_dataframe["global_tilted_irradiance"]

print(gti_df)

In [ ]:
TYPES = ("hydro", "eolienne", "solaire")
import pandas as pd

df = pd.read_csv("data/raw/prod_hydro.csv")

def clean(df: pd.DataFrame, energy_type: str) -> pd.DataFrame:
      df.iloc[:, 1] = df.iloc[:, 1].abs()
      if energy_type not in TYPES:
        print(f"Please enter any of types in this list : {TYPES} ")
      elif energy_type == "hydro":
        df = df.loc[(df.iloc[:, 1] <= 200)  & (df.iloc[:, 1] > 0)].copy()
        df = df.rename(columns={"date_obs_elab" : "date"})
      elif energy_type == "eolienne":
        df = df.loc[(df.iloc[:, 1] <= 100)  & (df.iloc[:, 1] > 0)].copy()
      elif energy_type == "solaire":
        df = df.loc[(df.iloc[:, 1] <= 100)  & (df.iloc[:, 1] > 0)].copy()
        df.iloc[:, 1] = df.iloc[:, 1]*1.5
      df = df.drop_duplicates(subset="date", keep="last")
      df = df.sort_values(by="date")
      df["date"] = pd.to_datetime(df["date"])
      df = df.dropna().reset_index(drop=True)
      df["date"] = df["date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")
      return df

df_clean = clean(df, "hydro")
display(df_clean)
records = df.to_dict(orient="records")
print(records)

# df_clean.loc[df_clean["prod_hydro"].duplicated() == True]
# df_clean


In [ ]:
import os

if os.access(os.getcwd()+"/data/train", os.F_OK) is False:
    os.makedirs(str(os.getcwd()) + "/data/train")

type(os.getcwd())

In [ ]:
from supabase import create_client
import os
import pandas as pd
from dotenv import load_dotenv
import json
from sqlalchemy import create_engine, MetaData, Table, Column, Float, Integer, DateTime, text
from sqlalchemy.pool import NullPool

load_dotenv()
types = ("hydro", "solaire", "eolienne")
TYPES = os.getenv("types")
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")
URL = os.getenv("SUPABASE_URL")
SERVICE_ROLE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"

client = create_client(URL, SERVICE_ROLE_KEY)
engine = create_engine(DATABASE_URL, poolclass=NullPool)
meta = MetaData()

def fetch_data(energy_type:str, file_path:str = os.path.join(os.getcwd(), "data/train/")):
        if os.access(file_path, os.F_OK) is False:
            os.makedirs(file_path)
        if energy_type in (None, "solaire"):
            with engine.begin() as conn:
                solaire_table = Table("Solaire_data", meta, autoload_with=engine)
                result_solaire = conn.execute(solaire_table.select())
                df_solaire = pd.DataFrame(sorted(result_solaire))
                return df_solaire.to_csv(file_path+"solaire_train.csv")


                

            # data_solaire = (client.table("Solaire_data").select("*").order("date", desc=False).execute())
            # print(data_solaire.data[1])
            # for row in range(len(data_solaire.data)):
            #     df_solaire = pd.DataFrame(data_solaire.data, columns = data_solaire.data[row].keys())
            #     df_solaire = df_solaire.sort_values(by="id").reset_index(drop=True)
            # return df_solaire
            # df_solaire.to_csv(file_path+"solaire_train.csv")
        if energy_type in (None, "eolienne"):
            data_eolienne = (client.table("Eolienne_data").select("*").execute())
            df_eolienne = pd.DataFrame(data_eolienne)
            df_eolienne.to_csv(file_path+"eolienne_train.csv")
        if energy_type in (None, "hydro"):
            data_hydro = (client.table("Hydro_data").select("*").execute())
            df_hydro = pd.DataFrame(data_hydro)
            df_hydro.to_csv(file_path+"hydro_train.csv")

fetch_data("solaire")

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 43.6109,
	"longitude": 3.8763,
	"start_date": "2016-09-01",
	"end_date": "2025-09-25",
	"hourly": ["global_tilted_irradiance", "temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m", "wind_direction_10m"],
	"tilt": 35,
}
responses = openmeteo.weather_api(url, params=params)
response = responses[0]
hourly = response.Hourly()

hourly_global_tilted_irradiance = hourly.Variables(0).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

print(hourly_data)
# def load_api(energy_type, url, parameters:list) -> pd.DataFrame:
        
#     if energy_type == "solaire":
#         all_data = []
#         for param in parameters:
#             params = {"grandeur_solaire_elab": param, "size": 1500}
#             response = requests.get(url, params=params)
#             df = pd.DataFrame(response.json().get("data", []))
#             if not df.empty:
#                 df["grandeur_solaire_elab"] = param
#                 all_data.append(df)
#         if all_data:
#             return pd.concat(all_data, ignore_index=True)
#         return pd.DataFrame()

# load_api("solaire", url, ["global_tilted_irradiance"])

In [ ]:
import requests
import pandas as pd
import numpy as np

url = "https://hubeau.eaufrance.fr/api/v2/hydrometrie/observations_tr"
parameters = {
    "code_entite": "Y321002101",
    "grandeur_hydro_elab": ["QmnJ", "HIXnJ"],
    "date_debut_obs": "2022-09-01",
    "date_fin_obs": "2025-09-29",
    "size": 1800,
    "page":1
}

response = requests.get(url, params=parameters)
response.raise_for_status()

data = response.json().get("data", [])
df = pd.DataFrame(data)

def clean(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        print("Aucune donnée API")
        return df

    date_col = "date_obs_elab" if "date_obs_elab" in df.columns else "date_obs"
    g_col    = "grandeur_hydro_elab" if "grandeur_hydro_elab" in df.columns else "grandeur_hydro"
    v_col    = "resultat_obs_elab" if "resultat_obs_elab" in df.columns else "resultat_obs"

    print("Dates min/max :", df[date_col].min(), "→", df[date_col].max())
    print("Grandeurs uniques :", df[g_col].dropna().unique()[:20])

    missing = {date_col, g_col, v_col} - set(df.columns)
    if missing:
        raise ValueError(f"Colonnes manquantes: {missing}")

    df = df[[date_col, g_col, v_col]].copy()
    df["date"] = pd.to_datetime(df[date_col], errors="coerce").dt.normalize()

    s = df[g_col].astype("string").str.upper().str.strip()
    df["norme"] = pd.Series(pd.NA, index=df.index, dtype="string")
    df.loc[s.str.startswith("Q"), "norme"] = "QmnJ"
    df.loc[s.str.startswith("H"), "norme"] = "HIXnJ"

    df = df[df["norme"].notna() & df["date"].notna()].copy()

    if df.empty:
        return pd.DataFrame(columns=["date", "QmnJ", "HIXnJ"])

    wide = (
        df.pivot_table(
            index="date",
            columns="norme",
            values=v_col,
            aggfunc="mean",
        )
        .reset_index()
    )

    for c in ["QmnJ", "HIXnJ"]:
        if c not in wide.columns:
            wide[c] = np.nan
    wide[["QmnJ", "HIXnJ"]] = wide[["QmnJ", "HIXnJ"]].apply(pd.to_numeric, errors="coerce")

    wide = wide.sort_values("date").reset_index(drop=True)
    wide["date"] = wide["date"].dt.strftime("%Y-%m-%d")

    return wide[["date", "QmnJ", "HIXnJ"]]

print(clean(df))

In [ ]:
from datetime import date

all_data = []
grandeurs = ["QmJ", "HIXnJ"]
url = "https://hubeau.eaufrance.fr/api/v2/hydrometrie/obs_elab"
code_entite = "Y321002101"

def load_api() -> pd.DataFrame:
        for grandeur in grandeurs:
            params = {
                      "code_entite": code_entite, 
                      "grandeur_hydro_elab": grandeur, 
                      "date_debut_obs" : "2022-09-01",
                      "date_fin_obs": "2025-09-29",
                      "size": 1500,}
            response = requests.get(url, params=params)
            response.raise_for_status()
            df = pd.DataFrame(response.json().get("data", []))
            if not df.empty:
                df["grandeur_hydro_elab"] = grandeur
                all_data.append(df)
        if all_data:
            return pd.concat(all_data, ignore_index=True)
        return df

def load_api2():
    for grandeur in grandeurs:
        params = {
            "code_entite": code_entite, 
            "grandeur_hydro_elab": grandeur, 
            "date_debut_obs_elab" : "2022-09-01",
            "date_fin_obs_elab": "2025-09-29",
            "size": 1500,}
        response = requests.get(url, params=params)
        response.raise_for_status()
        df = pd.DataFrame(response.json().get("data", []))
    if not df.empty:
        df["grandeur_hydro_elab"] = grandeur
        all_data.append(df)
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    return df

def clean(df: pd.DataFrame,
          start="2022-01-01",
          end=None,
          expected_cols=("QmJ", "HIXnJ")) -> pd.DataFrame:

    if df.empty:
        print("Aucune donnée API")
        end = end or date.today().isoformat()
        idx = pd.date_range(start, end, freq="D")
        out = pd.DataFrame(index=idx).reset_index().rename(columns={"index": "date"})
        out.insert(0, "id", range(1, len(out) + 1))
        return out
    
    df = df.copy()
    df["date"] = pd.to_datetime(df["date_obs_elab"]).dt.normalize()
    df = df[["date", "grandeur_hydro_elab", "resultat_obs_elab"]]

    df_pivot = (
        df.pivot_table(
            index="date",
            columns="grandeur_hydro_elab",
            values="resultat_obs_elab",
            aggfunc="mean"
        )
        .sort_index()
    )

    end = end or date.today().isoformat()
    full_idx = pd.date_range(start, end, freq="D")
    df_pivot = df_pivot.reindex(full_idx)

    present_cols = [c for c in expected_cols if c in df_pivot.columns]
    if not present_cols:
        out = df_pivot.reset_index().rename(columns={"index": "date"})
        out.insert(0, "id", range(1, len(out) + 1))
        return out

    df_pivot = df_pivot[present_cols]
    for col in df_pivot.columns:
        df_pivot[col] = pd.to_numeric(df_pivot[col], errors="coerce")

    bounds_max = {
        "QmJ": 10000,
        "HIXnJ": 2000 
    }
    for col in df_pivot.columns:
        df_pivot[col] = df_pivot[col].where(df_pivot[col] > 0)
        if col in bounds_max:
            df_pivot[col] = df_pivot[col].where(df_pivot[col] < bounds_max[col])

    for col in df_pivot.columns:
        series = df_pivot[col].dropna()
        if series.empty:
            continue
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        low = Q1 - 1.5 * IQR
        high = Q3 + 1.5 * IQR
        df_pivot[col] = df_pivot[col].where((df_pivot[col] >= low) & (df_pivot[col] <= high))

    out = df_pivot.reset_index().rename(columns={"index": "date"})
    out = out[["date","QmJ", "HIXnJ"]].fillna(0)

    return out

df = load_api2()
df_clean = clean(df)
df_clean

HTTPError: 400 Client Error: Bad Request for url: https://hubeau.eaufrance.fr/api/v2/hydrometrie/obs_elab?code_entite=Y321002101&grandeur_hydro_elab=QmJ&date_debut_obs_elab=2022-09-01&date_fin_obs_elab=2025-09-29&size=1500

In [7]:
from supabase import create_client, Client
from abc import ABC, abstractmethod
from dotenv import load_dotenv
from retry_requests import retry
from datetime import date
import pandas as pd
import requests
import openmeteo_requests
import requests_cache
import os


cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

api_url = "https://archive-api.open-meteo.com/v1/archive"

params = {
"latitude": 43.6109,
"longitude": 3.8763,
"start_date": "2016-09-01",
"end_date": "2025-09-25",
"hourly": ["global_tilted_irradiance", "temperature_2m"],
"tilt": 35,
}
responses = openmeteo.weather_api(api_url, params=params)
hourly = responses[0].Hourly()
hourly_global_tilted_irradiance = hourly.Variables(0).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
freq = pd.Timedelta(seconds = hourly.Interval()),
inclusive = "left"
)}
hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_dataframe = pd.DataFrame(data = hourly_data)
daily_dataframe= hourly_dataframe.resample('D', on='date').sum()
daily_dataframe = daily_dataframe.reset_index().rename(columns={"index": "date"})
daily_dataframe

,date,global_tilted_irradiance,temperature_2m
0,2016-09-01 00:00:00+00:00,6863.279785,626.812012
1,2016-09-02 00:00:00+00:00,7161.439453,617.012024
2,2016-09-03 00:00:00+00:00,6725.481445,608.161987
3,2016-09-04 00:00:00+00:00,6229.298828,633.661987
4,2016-09-05 00:00:00+00:00,6940.680664,640.262024
...,...,...,...
3307,2025-09-21 00:00:00+00:00,2806.606689,500.795990
3308,2025-09-22 00:00:00+00:00,5779.825684,435.496002
3309,2025-09-23 00:00:00+00:00,5843.613770,358.295990
3310,2025-09-24 00:00:00+00:00,5426.826172,348.295990


In [ ]:
if self.energy_type == "hydro":
        df = df.loc[(df.iloc[:, 1] <= 200)  & (df.iloc[:, 1] > 0)].copy()
        df = df.rename(columns={"date_obs_elab" : "date"})
      elif self.energy_type == "eolienne":
        df = df.loc[(df.iloc[:, 1] <= 100)  & (df.iloc[:, 1] > 0)].copy()
      elif self.energy_type == "solaire":
        df = df.loc[(df.iloc[:, 1] <= 100)  & (df.iloc[:, 1] > 0)].copy()
        df.iloc[:, 1] = df.iloc[:, 1]*1.5
      df["date"] = pd.to_datetime(df["date"])
      df = df.sort_values(by="date")
      df = df.drop_duplicates(subset="date", keep="first")
      df = df.dropna().reset_index(drop=True)
      df["date"] = df["date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")
      return df